In [47]:
import sys
sys.path.insert(0, "D:\\Documents\\food_recipe_gen\\recipe_1m_analysis")
import os
import pandas as pd
import torch
import utils

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
FOLDER_PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\scirep-cuisines-detail"
FILE = "cleaned_data.pkl"

In [132]:
df=pd.read_pickle(os.path.join(FOLDER_PATH,FILE))
df=df.reset_index()

In [34]:
vocab_ingrs = utils.Vocabulary()
for ingredients in df.loc[:,"ingredients"]:
    for ingr in ingredients:
        vocab_ingrs.add_word(ingr)
vocab_ingrs.word2idx

{'egg': 0,
 'yeast': 1,
 'wheat': 2,
 'milk': 3,
 'lard': 4,
 'pork': 5,
 'carrot': 6,
 'pea': 7,
 'onion': 8,
 'potato': 9,
 'maple_syrup': 10,
 'almond': 11,
 'honey': 12,
 'oat': 13,
 'date': 14,
 'vegetable_oil': 15,
 'whole_grain_wheat_flour': 16,
 'butter': 17,
 'lovage': 18,
 'clam': 19,
 'thyme': 20,
 'black_pepper': 21,
 'parsley': 22,
 'ginger': 23,
 'bay': 24,
 'celery': 25,
 'cinnamon': 26,
 'mustard': 27,
 'cane_molasses': 28,
 'raisin': 29,
 'cream': 30,
 'asparagus': 31,
 'olive_oil': 32,
 'pepper': 33,
 'garlic': 34,
 'tomato': 35,
 'cilantro': 36,
 'tea': 37,
 'jasmine': 38,
 'vegetable': 39,
 'brown_rice': 40,
 'lemon_juice': 41,
 'soy_sauce': 42,
 'white_wine': 43,
 'chicken': 44,
 'vanilla': 45,
 'rice': 46,
 'mushroom': 47,
 'chicken_broth': 48,
 'basil': 49,
 'porcini': 50,
 'mozzarella_cheese': 51,
 'tuna': 52,
 'lemon': 53,
 'beef': 54,
 'fish': 55,
 'cocoa': 56,
 'green_bell_pepper': 57,
 'oregano': 58,
 'rosemary': 59,
 'coffee': 60,
 'banana': 61,
 'squash': 

In [63]:
vocab_cuisine = utils.Vocabulary()
for cuisine in df.loc[:,"cuisine"]:
    vocab_cuisine.add_word(cuisine)

In [67]:
INPUT_SIZE=len(vocab_ingrs)
NUM_CLASSES = len(vocab_cuisine)
HIDDEN_SIZE = 100

In [161]:
def ingr2idx(ingr_list):
    input_=[]
    for ingr in ingr_list:
        input_.append(vocab_ingrs.word2idx[ingr])
    return F.one_hot(torch.Tensor(input_).to(torch.int64), INPUT_SIZE)

class RecipesDataset(Dataset):
    """Recipes dataset for cuisine classification. Only from ingredients for now"""

    def __init__(self, dataframe):
        """
        Args:
            file (string): Path to the file
        """
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        a=vocab_cuisine.word2idx[self.data.loc[idx,"cuisine"]]
        b=ingr2idx(self.data.loc[idx,"ingredients"])
        sample = {'label': a ,
                  'ingredients':b }
        

        return sample

In [162]:
vocab_cuisine.word2idx[df.loc[5000,"cuisine"]]

31

In [184]:
dataset = RecipesDataset(df)
loader = DataLoader(dataset, batch_size=1)#,shuffle=True)

In [185]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = F.relu(self.layer_1(x))
        out = F.relu(self.layer_2(out))
        out = self.output_layer(out)
        return out

net = Net(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)

In [186]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [187]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data["ingredients"]
        labels = data["label"]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'